In [2]:
ls ../../testdata/bam

001_A.bam                  002_A.chr7.bam.bai
001_A.bam.bai              002_B.bam
001_A.chr1.bam             002_B.bam.bai
001_A.chr1.bam.bai         002_B.chr7.bam
001_A.chr7.bam             002_B.chr7.bam.bai
001_A.chr7.bam.bai         003_A.bam
001_A.chr7.chr7.bam        003_A.bam.bai
001_A.chr7.chr7.bam.bai    003_A.chr1.bam
001_B.bam                  003_A.chr1.bam.bai
001_B.bam.bai              003_A.chr1.chr1.40018.bed
001_B.chr7.bam             003_A.chr1.chr1.40019.bed
001_B.chr7.bam.bai         003_A.chr7.bam
002_A.bam                  003_A.chr7.bam.bai
002_A.bam.bai              003_B.bam
002_A.chr1.bam             003_B.bam.bai
002_A.chr1.bam.bai         003_B.chr7.bam
002_A.chr7.bam             003_B.chr7.bam.bai


In [ ]:
ls ../data

In [4]:
import sys
import os
sys.path.append('../code')
from ebrun import run_eb
# set the paths
testdata = '../../testdata'

EBparams = {
    'MAPQ': 20,
    'Q': 25,
    'fitting_penalty': 0.5
}
  
run_eb(
    table='../data/002.csv',
    tumor_bam=f"{testdata}/bam/002_B.chr7.bam",
    output=['../output/002.chr7.EB'],
    pon_list='../testpon/Pon_chr7.txt',
    chrom="chr7",
    log='..output/log.txt',
    threads=8,
    EBparams=EBparams,
    full_output=False,
    # import the scripts
    cleanpileup='../shell/cleanpileup.mawk',
    csv2bed='../shell/csv2bed.mawk',
    pon2cols='../shell/pon2cols.mawk',
    pile2count='../shell/pile2count.mawk',
    matrix2EBinput='../shell/matrix2EBinput.mawk',
    makeponlist='../shell/makeponlist.sh',
    )

15:41:48 : Computing EBscore for chrom chr7
15:41:48 : Piling up chr7 of ../../testdata/bam/002_B.chr7.bam with Pon List.
$ samtools mpileup -B -q 20 -Q 25 -l ../output/002.chr7.bed -r chr7 -b ../output/002.chr7.pon | cut -f $(../shell/pon2cols.mawk < ../output/002.chr7.pon) | ../shell/cleanpileup.mawk | ../shell/pile2count.mawk > ../output/002.chr7.matrix
15:44:55 : Pileup matrix for chrom chr7 of ../../testdata/bam/002_B.chr7.bam completed.
15:44:57 : Created EBscore for chrom chr7 of ../../testdata/bam/002_B.chr7.bam and written to ../output/002.chr7.EB


In [ ]:

from io import StringIO
from subprocess import Popen, PIPE, run
from HDR_run import HDR_master

home = '/Users/mahtin'
path = '/Users/martinscience/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/001.chr1.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
genome_split_path = os.path.join(home,'Dropbox/Icke/Work/static/genome/gatk/hg38/split')
HDR_config = {
    "minAltSum": 2,
    "minAltRatio": 0.1,
    "maxAltRatio": 0.85,
    "MINQ": 15,
    "MINSIM": .50,
    "PAD": 100,
    "MINq": 10,
    "MinAltSupport": 5,
    "MinHDRCount": 1,
    "pile2hotspot": '../shell/pile2hotspot.mawk',#
    "pile2hotspot_chrom": '../shell/pile2hotspot_chrom.mawk',
    "editbam": '../shell/editbam.mawk',
    "bam2csv": '../shell/bam2csv.mawk',
    "genome_split_path": genome_split_path # the path to the folder with chrom-split genomes (chr1.fa..)
}
HDR_master(mut_file, bam_file=bam_file, pileup_file=pileup_file, chrom=chrom, threads=threads, HDR_config=HDR_config)

In [ ]:
ls data

In [ ]:

home = '/Users/mahtin'
path = '/Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/002.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
mut_df = pd.read_csv(mut_file, sep='\t').loc[:, [
        'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene']]
# make Chr column categorical for sorting .. and sort
chrom_list = [f"chr{i}" for i in range(23)] + ['chrX', 'chrY']
mut_df['Chr'] = pd.Categorical(mut_df['Chr'], chrom_list)
mut_df = mut_df.sort_values(['Chr', 'Start'])
mut_df = mut_df.query('Chr == @chrom')
mut_df
mut_split = np.array_split(mut_df, 4)

In [ ]:
mut_df = mut_split[0]

In [ ]:
anno_df = mut_df.sort_values(['Chr', 'Start']).iloc[0:,:5]
anno_df

In [ ]:
def reduce_regions(df, padding):
    '''
    takes a mutation list and returns a region list using padding
    overlapping regions are reduced to one using the gap strategy
    '''

    df = df.sort_values('Start')
    df['Start'] = df['Start'] - padding
    df['End'] = df['End'] + padding
    # find the break points
    # if Start is greater than previous End (using shift), this is a gap --> df['gap'] = 1
    df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
    # id different reads according to gap
    # cumulative sum does not increase at df['gap'] == 0 and so these consecutive stretches are grouped together
    df['gap'] = df['gap'].cumsum()
    # groupby the coverage break group and condense individual coverage islands
    # agg has to contain the neccessary shared columns TransLength because it is needed for coverage computation
    df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'})
    return df.reset_index('gap').drop(columns='gap')

In [ ]:
df = anno_df
anno_df
padding = 100
df['Start'] = df['Start'] - padding
df['End'] = df['End'] + padding
df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
df['gap'] = df['gap'].cumsum()
df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'}).reset_index().drop(columns='gap')
df

In [ ]:
bed_df = reduce_regions(anno_df, 100)
bed_df

In [ ]:
pd.DataFrame(columns=[
        'Chr',
        'Start',
        'End',
        'Ref',
        'Alt',
        'Gene',
        'HDRcand',
        'HDRcount',
        'HDRinfo',
    ])